# Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

체인은 일반적으로 LLM(Large Language Model)과 프롬프트를 결합하며, 이런 빌딩 블록들을 여러 개 함께 사용하여 텍스트나 다른 데이터에 대해 일련의 작업을 수행할 수 있습니다. 

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

In [3]:
#!pip install pandas

In [4]:
import pandas as pd
df = pd.read_csv('Data.csv')

이전과 같이 환경 변수를 로드할 것입니다.

그런 다음 우리가 사용할 데이터도 로드할 것입니다. 이 체인들의 힘 중 하나는 한 번에 많은 입력을 처리할 수 있다는 것입니다. 여기서 우리는 판다스 DataFrame을 로드합니다.

In [5]:
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...


판다스 DataFrame은 여러 다른 데이터 요소를 포함하는 데이터 구조입니다. 판다스에 익숙하지 않아도 걱정하지 마세요. 주요한 점은 나중에 사용할 수 있도록 어떤 데이터를 로딩하는 것입니다. 그래서 이 판다스 DataFrame 안을 살펴보면, 제품 열과 리뷰 열이 있는 것을 볼 수 있습니다. 그리고 각각의 행들은 우리가 체인들로 전달하고 시작할 수 있는 다른 데이터 포인트들입니다.

## LLMChain

In [6]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

우리가 처음으로 다루게 될 체인은 LLM 체인입니다. 이것은 단순하지만 꽤 강력한 체인으로, 앞으로 소개될 많은 체인들의 기반이 됩니다.

In [7]:
llm = ChatOpenAI(temperature=0.9)


그래서, 우리는 세 가지 서로 다른 것들을 가져올 예정입니다. OpenAI 모델, 즉 LLM을 가져옵니다. 그런 다음 채팅 프롬프트 템플릿을 가져옵니다 - 이것이 바로 프롬프트 입니다.

In [8]:
prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

그 후 LLM 체인을 가져옵니다.

그래서 먼저, 우리가 사용하려는 언어 모델 초기화 작업부터 시작합니다.
따라서 재미있는 설명문 생성 가능성이 높아지도록 오픈AI와 함께 고온도 설정으로 채팅 모델 초기화를 진행하겠습니다.

이제 프롬프트를 초기화하겠습니다. 이 프롬프트는 '제품'이라는 변수를 가져와서 LLM에 그 제품을 만드는 회사에 가장 적합한 이름은 무엇인지 생성하도록 요청합니다. 마지막으로, 이 두 가지를 체인으로 결합합니다.

그래서, 이것을 우리가 LLM 체인이라고 부릅니다.

매우 간단합니다. 그냥 LLM과 프롬프트의 결합입니다. 하지만 이제 이 체인은 프롬프트를 통해 LLM로 순차적으로 실행할 수 있게 해줍니다. 따라서 제품명이 queen-size-sheet-set라면 chain.run을 사용하여 체인을 실행할 수 있습니다.

In [9]:

chain = LLMChain(llm=llm, prompt=prompt)

In [10]:
product = "Queen Size Sheet Set"
chain.run(product)

'Royal Beddings.'

그러면 이것은 백엔드에서 프롬프트 형식을 지정한 후 전체 프롬프트를 LLM로 전달합니다. 그래서 우리는 Royal Bettings라는 가상 회사의 이름을 반환받았다는 것을 알 수 있습니다.

## SimpleSequentialChain

In [11]:
from langchain.chains import SimpleSequentialChain

순차 체인은 하나 뒤에 하나를 연결하여 실행하는 일련의 체인입니다.   
시작하려면 간단한 순차 체인을 가져와야 합니다. 이것은 하위 체인들이 단일 입력만 기대하고 단일 출력만 반환할 때 잘 작동합니다.

In [12]:
llm = ChatOpenAI(temperature=0.9)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

순차 체인은 하나 뒤에 하나를 연결하여 실행하는 일련의 체인입니다. 시작하려면 간단한 순차 체인을 가져와야 합니다. 이것은 하위 체인들이 단일 입력만 기대하고 단일 출력만 반환할 때 잘 작동합니다.

In [13]:

# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following \
    company:{company_name}"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

그런 다음 두 번째 체인을 만듭니다.
이 두 번째 체인에서는 회사 이름을 가져와서 그 회사에 대한 20단어 설명문을 출력할 것입니다.

따라서 이러한 체인들은 서로 연속적으로 실행되어야 할 가능성이 있는데, 여기에서는 처음의 결과(회사 이름)가 두 번째의 입력으로 전달됩니다. 간단한 순차적 연결로 이를 쉽게 수행할 수 있습니다. 여기에 우리가 위에서 설명한 두 개의 연결체가 있습니다.
우리는 이 전체적으로 간단한 연결체를 호출합니다.

In [14]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [15]:
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
Royal Rest Linens
Royal Rest Linens supplies high-quality, luxurious bedding and linens to hotels, resorts, and residential clients. Exceptional comfort and durability guaranteed.

> Finished chain.


'Royal Rest Linens supplies high-quality, luxurious bedding and linens to hotels, resorts, and residential clients. Exceptional comfort and durability guaranteed.'

그런 다음 두 번째 체인을 만듭니다.
이 두 번째 체인에서는 회사 이름을 가져와서 그 회사에 대한 20단어 설명문을 출력할 것입니다.

따라서 이러한 체인들은 서로 연속적으로 실행되어야 할 가능성이 있는데, 여기에서는 처음의 결과(회사 이름)가 두 번째의 입력으로 전달됩니다. 간단한 순차적 연결로 이를 쉽게 수행할 수 있습니다. 여기에 우리가 위에서 설명한 두 개의 연결체가 있습니다.
우리는 이 전체적으로 간단한 연결체를 호출합니다.

하지만 여러 개의 입력 혹은 여러 개의 출력들 중 언제든지 선택해야 할 때엔 어떻게 해야 할까요?
우리는 정규 순착적 연결체를 사용하여 이를 수행할 수 있습니다.

## SequentialChain

In [16]:
from langchain.chains import SequentialChain

하지만 여러 개의 입력 혹은 여러 개의 출력들 중 언제든지 선택해야 할 때엔 어떻게 해야 할까요?
우리는 정규 순착적 연결체를 사용하여 이를 수행할 수 있습니다.

In [17]:
llm = ChatOpenAI(temperature=0.9)

# prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to english:"
    "\n\n{Review}"
)
# chain 1: input= Review and output= English_Review
chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key="English_Review"
                    )


그래서 첫 번째 연결체에서는, 우리는
리뷰를 가져와 영어로 번역합니다.

In [18]:
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{English_Review}"
)
# chain 2: input= English_Review and output= summary
chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key="summary"
                    )


두 번째 연결체에서는, 우리는 그 리뷰의 요약문을 한 문장으로 만듭니다.
이것은 앞서 생성된 영어 리뷰를 사용합니다.

In [19]:
# prompt template 3: translate to english
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="language"
                      )


세 번째 연결체는 원래의 리뷰 언어가 무엇인지 감지하는 작업을 합니다.
따라서 여기에서 이것은 원래의 리뷰에 의해 사용되며,
마침내 네 번째 연결체는 여러 개의 입력을 받아 들일 것입니다.
딱히 이것은 요약 변수,
우리가 두 번째 연결체로 계산한 변수,언어 변수를 받아들입니다.

요약문에 대한 후속 응답을 지정된 언어로 요청할 것입니다.

이 모든 하위 체인들에 대해 주의 깊게 알아야 할 중요한 점은 입력 키와 출력 키가 매우 정확해야 한다는 것입니다. 여기서, 우리는 리뷰를 가져옵니다. 이것은 시작할 때 전달될 변수입니다.

우리는 출력 키를 명시적으로 영어 리뷰로 설정할 수 있습니다. 이것은 다음 프롬프트에서 사용되며, 아래에서 영어 리뷰라는 같은 변수 이름으로 가져옵니다.

그 체인의 출력 키를 요약으로 설정하면, 마지막 체인에서 사용되는 것을 볼 수 있습니다.
세 번째 프롬프트는 원래의 변수인 리뷰를 가져와서,
언어라고 출력합니다,
이것 또한 최종 프롬프트에서 다시 사용됩니다.
이러한 많은 입력과 출력들 때문에 이러한 변수 이름들이 정확하게 일치하는지 확인하는 것이 매우 중요합니다. 만약 당신이 어떤 키 에러를 받으면, 반드시
그들이 정렬되었는지 확인해야 합니다.

In [20]:

# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)
# chain 4: input= summary, language and output= followup_message
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message"
                     )


In [21]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review", "summary","followup_message"],
    verbose=True
)

In [22]:
review = df.Review[5]
overall_chain(review)



> Entering new SequentialChain chain...

> Finished chain.


{'Review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?",
 'English_Review': "I find the taste mediocre. The foam doesn't hold, it's weird. I buy the same ones in the store and the taste is much better... Old stock or counterfeit!?",
 'summary': "The reviewer found the taste of the product mediocre and noticed that the foam doesn't hold, questioning if it is old stock or counterfeit.",
 'followup_message': "Le critique a trouvé le goût du produit médiocre et a remarqué que la mousse ne tient pas, se demandant s'il s'agit d'un vieux stock ou d'une contrefaçon. Il recommande de faire attention avant d'acheter ce produit et de vérifier la date d'expiration."}

간단한 순차적 연결체는 여러 개의 연결체를 받아 들여,
각각 하나씩 질문과 답변을 가집니다.
이것의 시각적 설명을 보려면 슬라이드에서 보실 수 있는 그림을 참조하세요,
여기서 한 개의 연결체가
다른 연결체 뒤에 오도록 배치되어 있습니다.
여기서 우리는 순차적 연결체의 시각적 설명을 볼 수 있습니다.
위에 있는 연결체와 비교해서 어느
연결체 단계에서든지 여러 개의 입력 변수들을 가져올 수 있다는 점에 유념하세요.
복잡한 다운스트림
연결체가 필요하고 그것이 여러 개의
전 단계 연결체들로 구성되어야 할 때 유용합니다.

모든 이러한 연결체들이 준비되면, 우리는 쉽게 그것들을 순차적 연결체에 결합할 수 있습니다. 여기서 당신은 우리가 위에서 만든 네 개의 연결체들을
연결 변수로 전달하는 것을 알아챌 수 있습니다. 우리는 입력 변수를 하나의 인간 입력, 즉 리뷰와 함께 생성합니다.
그리고 나서 모든 중간 출력을 반환하려고 합니다.
따라서 영어 리뷰, 요약문, 그리고 후속 메시지 등입니다.

이제 이것을 어떤 데이터 위에서 실행할 수 있습니다.
따라서 리뷰를 선택하고 전반적인 체인을 통해 그것을 전달합시다.
여기서 원래의 리뷰가
프랑스어로 보입니다.
우리는 번역된 영어 리뷰를 볼 수 있으며, 그 리뷰의 요약문과 원래 언어인 프랑스어로 된 후속
메시지를 볼 수 있습니다.

당신은 여기에서 비디오를 잠시 멈추고 다른 입력값들을 넣어보아야 합니다. 지금까지 우리는 LLM 체인과 순차 체인에 대해 다루었습니다. 하지만 더 복잡한 작업을 해야 한다면 어떻게 해야 할까요?

## Router Chain

매우 일반적이지만 기본적인 작업은
입력에 딱 맞는 체인으로 입력값들을 라우팅하는 것입니다.
이것은 여러 개의 하위 체인 간에 라우팅하는 것으로 상상할 수 있는 좋은 방법으로, 각각이 특정 타입의 입력에 특화되어 있다면,
당신은 라우터 체인이 어느 하위 체인으로 전달해야 하는지 먼저 결정하도록 만들 수 있으며, 그럼 이후 해당 체인으로 전달됩니다.

In [23]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

구체적 예제로 보자면 주제에 따라 다른 유형의 연결체 사이에서 경로를 설정한다고 가정해보세요. 여기에서 서로 다른 프롬프트가 있는데요, 한 프롬프트는 물리학에 관한 질문에 대답하는데 좋고, 두 번째 프롬프트는 수학 질문에 대답하는데 좋으며, 세 번째는 역사, 그리고 네 번째는 컴퓨터 과학에 대한 것입니다. 이러한 프롬프트 템플릿을 모두 정의해 봅시다.

In [24]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

각각의 프롬프트 템플릿은 이름과 설명을 포함합니다. 예를 들어, 물리학에 대한 설명은 "물리학 질문에 대답하는 것"입니다. 이 정보는 라우터 체인이 어느 하위 체인을 사용할지 결정하는 데 사용됩니다.

다른 유형의 체인들도 필요합니다. 여기서 필요한 것은 멀티-프롬프트 체인입니다.
멀티-프롬프트 체인은 여러 개의 다른 프롬프트 템플릿 사이에서 경로를 설정할 때 사용되는 특정 유형의 체인입니다.
하지만 이것은 당신이 경로 설정 가능한 한 가지 유형일 뿐입니다.
당신은 언제든지 양쪽 모두가 연결체라면 양쪽 간에 경로를 설정할 수 있습니다.

LLM(언어 모델)라우터 연결체도 가져와야 합니다.
LLM(언어 모델)라우터 연결체는 언어 모델 자체를 사용하여
다른 하위 연결체 사이에서 경로를 설정합니다. 이곳에서 위에서 제공된 설명과 이름이 사용됩니다.

In [25]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [26]:
llm = ChatOpenAI(temperature=0)

우선, 우리가 사용할 언어 모델을 가져오고 정의합니다.

그런 다음 목적지 연결체들(라우터 체인에 의해 호출될) 을 생성합니다. 보시다시피, 각 목적지 연결체 자체가 LLM(언어 모델) 연결체 입니다.

In [27]:

destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

목적지 연결체 외에도 기본(default)연결체도 필요합니다. 이것은 라우터가 어느 하위 연결체를 사용해야 할지 결정할 수 없을 때 호출되는 연결체입니다. 위의 예제에서 이것은 입력 질문이 물리학, 수학, 역사, 컴퓨터 과학과 관련이 없을 때 호출될 것입니다.

그런 다음 LLM이 다른 체인 사이에서 경로를 설정하는데 사용하는 템플릿을 정의합니다.
여기에는 수행할 작업에 대한 지시사항과 출력 형식이 포함되어 있습니다.

몇 가지 조각들을 모아 라우터 체인을 만들어 보겠습니다.

In [28]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

첫째, 위에서 정의한 목적지와 함께 템플릿을 형식화하여 전체 라우터 템플릿을 생성합니다. 이 템플릿은 여러 다른 유형의 목적지에 유연하게 적용됩니다.
여기서 할 수 있는 일 중 하나는 잠시 멈추고 다른 종류의 목적지를 추가하는 것입니다.
따라서 여기서 물리학, 수학, 역사, 컴퓨터 과학 외에도 영어나 라틴어와 같은 다른 주제를 추가할 수 있습니다.

라우터 체인을 만들기 위해 이제 필요한 모든 요소들이 준비되었습니다. 우리는 LLM 라우터 체인을 생성하고, 목적지 연결체, 기본 연결체, 그리고 라우터 템플릿을 전달합니다. 이렇게 하면 우리는 입력에 따라 다른 하위 체인으로 경로를 설정할 수 있는 라우터 체인을 가지게 됩니다.

In [29]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

그럼 이제 이것이 어떻게 작동하는지 확인해봅시다. 예를 들어 "물리학"에 관한 질문을 하면, 라우터 체인은 입력 질문의 내용에 따라 적절한 목적지 연결체를 선택하고 그 연결체로 질문을 전달합니다.

In [30]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [31]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

In [32]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


"Black body radiation refers to the electromagnetic radiation emitted by a perfect black body, which is an object that absorbs all radiation that falls on it and emits radiation at all wavelengths. The radiation emitted by a black body depends only on its temperature and follows a specific distribution known as Planck's law. This type of radiation is important in understanding the behavior of stars, as well as in the development of technologies such as incandescent light bulbs and infrared cameras."

In [33]:
chain.run("what is 2 + 2")



> Entering new MultiPromptChain chain...
math: {'input': 'what is 2 + 2'}
> Finished chain.


'As an AI language model, I can answer this question easily. The answer to 2 + 2 is 4.'

In [34]:
chain.run("Why does every cell in our body contain DNA?")



> Entering new MultiPromptChain chain...
None: {'input': 'Why does every cell in our body contain DNA?'}
> Finished chain.


'Every cell in our body contains DNA because DNA carries the genetic information that determines the characteristics and functions of each cell. DNA contains the instructions for the synthesis of proteins, which are essential for the structure and function of cells. Additionally, DNA is responsible for the transmission of genetic information from one generation to the next. Therefore, every cell in our body needs DNA to carry out its specific functions and to maintain the integrity of the organism as a whole.'

그러나 만약 "음악"과 관련된 질문을 한다면 어떻게 될까요? 우리의 경우에서는 음악과 관련된 목적지가 없습니다. 따라서 라우터 체인은 기본 연결체로 경로를 설정할 것입니다.

여기서 중요한 점은 라우팅 로직이 상당히 유연하다는 것입니다. 당신은 자신만의 로직을 정의하거나 다양한 유형의 입력에 대응하는 여러 개의 하위 체인들 사이에서 경로를 설정할 수 있습니다.

많은 경우에서 복잡한 워크플로우는 여러 개의 순차적 및 병렬 연결체와 함께 사용되며, 각각이 서로 다른 유형의 작업에 최적화되어 있습니다. 당신은 이러한 도구들을 사용하여 매우 복잡하지만 강력한 워크플로우를 구축할 수 있습니다.

그럼 지금까지 설명드린 내용으로 충분히 흥미롭다면, 당신도 자신만의 LLM 체인과 순차/병렬/라우터 체인 등을 만들어보세요!